# **Visualization of MyAnimeList Anime Data**

Tasks of this notebook:
- Exploratory data analysis
- Visualizations

**1. Initial Setup** 

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px

In [39]:
# Defintion of functions and values of some constants
anime_data = pd.read_csv('anime_data_vis.csv')
anime_data.reset_index(inplace=True)

TODO List + Some ideas
- Animes with the longest and shortest duration per episode
- Number of animes per genres, type, rating, etc ....
- Average score, members, and favoruites per genre
- Average score, memebers, and favourites per season
- Average score based on soruce material
- score vs scored_by scatter plot, rank vs popularity scatter plot
- Distrubtiton of scores, memebers

- Average score and members per licensors, producers, studios

- Number of animes released per season and per year (2 Line Plots)
- Average score over time (Line Plot)
- Number of animes realeased per year differnated by the type (So we can see different trends)
- Pairplot score, scored_by, members, episodes, type
- Heatmap of anime realease frequency


- Word cloud of the most common words in a title
- Chord diagram of relationships between genres
- Treemap  or pie chart of animes by genres and type
- PCA 
-

# Basic Analysis

In [ ]:
print(f"Top 5 animes with the highest score:\n{anime_data.nlargest(5, columns=['score']).sort_values('score', ascending=False)[['title', 'score', 'rank']] }")
print(f"Top 5 animes with the lowest scores:\n{anime_data.nsmallest(5, columns=['score']).sort_values('score', ascending=False)[['title', 'score', 'rank']]}")
print(f"Top 5 animes with the highest number of members: \n {anime_data.nlargest(5, columns=['members']).sort_values('members', ascending=False)[['title', 'members', 'popularity']] }")
print(f"Top 5 animes with the lowest number of members: \n{anime_data.nsmallest(5, columns=['members']).sort_values('members', ascending=False)[['title', 'members', 'popularity']] }")

print(f"Top 10 animes with the most episodes:\n{anime_data.nlargest(10, 'episodes')[['title', 'episodes']]}") # LMAO One Piece doesnt crack the list
#

Top 5 animes with the highest score:
                                    title  score  rank
23013                   Sousou no Frieren   9.30   1.0
3960     Fullmetal Alchemist: Brotherhood   9.10   2.0
5664                          Steins;Gate   9.07   3.0
9873                             Gintama°   9.05   5.0
14838  Shingeki no Kyojin Season 3 Part 2   9.05   4.0
Top 5 animes with the lowest scores:
                              title  score     rank
10977                      Kokuhaku   2.41  14076.0
4193                   Tsui no Sora   2.32      NaN
387                 Hametsu no Mars   2.23  14077.0
6944             Utsu Musume Sayuri   2.00  14077.0
2925   Tenkuu Danzai Skelter+Heaven   1.89  14078.0
Top 5 animes with the highest number of members: 
                                   title  members  popularity
7425                 Shingeki no Kyojin  4170962           1
1393                         Death Note  4114852           2
3960   Fullmetal Alchemist: Brotherhood  3529544  